In [201]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from nesteddictionary import NestedDict 


from bs4 import BeautifulSoup  as bs
from pprint import pprint  
import requests  
import json

In [202]:
max_colwidth = 1000
pd.set_option("display.max_colwidth", max_colwidth)

In [203]:
!pip freeze > requirements.txt
RANDOM_SEED = 42

In [204]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

# Собираем датасет - парсим AUTO.RU

In [273]:
url = 'https://auto.ru/cars/used/sale/honda/civic/1101565623-563bf58d/'
url = 'https://auto.ru/cars/used/sale/skoda/octavia/1100575026-c780dc09/'

def parse_car_page(url):
    response = requests.get(url)
    response.encoding = 'utf-8'
    soup = bs(response.text, 'html.parser')
    color = soup.find('li', class_='CardInfoRow CardInfoRow_color').a.text
    engineDisplacement = soup.find('meta', itemprop="engineDisplacement")['content']
    fuelType = soup.find('meta', itemprop="fuelType")['content']



    s1 = soup.find('div', id='app').find('div', id="sale-data-attributes")['data-bem']
    d1 = json.loads(s1)['sale-data-attributes']

    s2 = soup.find('script', id="initial-state").string
    d2 = json.loads(s2)

    nested_dict = NestedDict( d2 )
    way_to_equip = nested_dict.findall('equipment')
    equipment_dict = dict(nested_dict[way_to_equip[0]])

    way_to_bodytype = nested_dict.findall('human_name')
    bodytype = nested_dict[way_to_bodytype[0]].lower()



    return dict({'brand':d1['mark'], 
                 'model_name':d1['model'], 
                 'productionDate':d1['year'],
                 'enginePower':d1['power'],
                 'image':d1['image'],
                 'mileage':d1['km-age'],
                 'equipment_dict': equipment_dict,
                 'bodyType': bodytype,
                 'color': color,
                 'engineDisplacement': engineDisplacement,
                 'fuelType': fuelType
                 })
    # return d
parse_car_page(url)

{'brand': 'SKODA',
 'model_name': 'OCTAVIA',
 'productionDate': 2014,
 'enginePower': 105,
 'image': 'https://avatars.mds.yandex.net/get-autoru-vos/1590523/8e9df38833e18d0b30b5540ae371860f/1200x900',
 'mileage': 74000,
 'equipment_dict': {'audiosystem-cd': True,
  'aux': True,
  '12v-socket': True,
  'condition': True,
  'computer': True,
  'cooling-box': True,
  'park-assist-r': True,
  'ashtray-and-cigarette-lighter': True,
  'tinted-glass': True,
  'abs': True,
  'isofix': True,
  'power-child-locks-rear-doors': True,
  'airbag-driver': True,
  'airbag-passenger': True,
  'airbag-side': True,
  'airbag-rear-side': True,
  'rain-sensor': True,
  'ptf': True,
  'high-beam-assist': True,
  'electro-mirrors': True,
  '16-inch-wheels': True,
  'engine-proof': True,
  'lock': True,
  'fabric-seats': True,
  'door-sill-panel': True,
  'front-seats-heat': True,
  'seat-transformation': True,
  'electro-window-back': True,
  'electro-window-front': True,
  'steel-wheels': True,
  'alarm': Tr

In [ ]:
response = requests.get(url)
response.encoding = 'utf-8'
soup = bs(response.text, 'html.parser')
s = soup.find('div', id='app').find('div', id="sale-data-attributes")['data-bem']
d = json.loads(s)['sale-data-attributes']
d

In [190]:
s = soup.find('script', id="initial-state").string
d = json.loads(s)
nested_dict = NestedDict( d )
way_to_bodytype = nested_dict.findall('human_name')
way_to_equip[0]
nested_dict[way_to_bodytype[0]].lower()
# d['card']['vehicle_info']['configuration']['human_name']
# ['card', 'vehicle_info', 'configuration', 'human_name']

'лифтбек'

In [265]:
soup.find('meta', itemprop="engineDisplacement")['content']

'1.2 LTR'

In [212]:
# print(soup.prettify())

In [ ]:
test = pd.read_csv('kaggle/test.csv')

In [271]:
l = list(test.columns)
l.remove('model_name')
l.remove('brand')
l.remove('productionDate')
l.remove('enginePower')
l.remove('image')
l.remove('mileage')
l.remove('equipment_dict')
l.remove('bodyType')
l.remove('color')
l.remove('engineDisplacement')
# l.remove('engineDisplacement')
# l.remove('engineDisplacement')
# l.remove('engineDisplacement')
# l.remove('engineDisplacement')
# l.remove('engineDisplacement')
# l.remove('engineDisplacement')
# l.remove('engineDisplacement')
l

['car_url',
 'complectation_dict',
 'description',
 'fuelType',
 'modelDate',
 'model_info',
 'name',
 'numberOfDoors',
 'parsing_unixtime',
 'priceCurrency',
 'sell_id',
 'super_gen',
 'vehicleConfiguration',
 'vehicleTransmission',
 'vendor',
 'Владельцы',
 'Владение',
 'ПТС',
 'Привод',
 'Руль',
 'Состояние',
 'Таможня']

In [272]:
s = test.loc[0]
s['engineDisplacement']
# json.loads(s)
# vehicleEngine
test['fuelType'].unique()

array(['бензин', 'дизель', 'гибрид', 'электро', 'газ'], dtype=object)